<a href="https://colab.research.google.com/github/busraguven/book-recommender/blob/main/book_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📚 Book Recommender Project — Starter Notebook
# Author: [Ulviye Busra Guven]
# Goal: Build a multimodal book recommender (text + metadata)

!pip install pandas numpy scikit-learn sentence-transformers faiss-cpu -q
import pandas as pd
import numpy as np


In [ ]:
data = [
    {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "genre": "Classic",
     "description": "A tragic story of wealth, love and the American dream."},
    {"title": "Pride and Prejudice", "author": "Jane Austen", "genre": "Romance",
     "description": "A witty exploration of manners, marriage and social standing."},
    {"title": "Dune", "author": "Frank Herbert", "genre": "Sci-Fi",
     "description": "An epic saga of politics, prophecy, and survival on the desert planet Arrakis."},
    {"title": "1984", "author": "George Orwell", "genre": "Dystopian",
     "description": "A chilling vision of a totalitarian future and the loss of freedom."},
    {"title": "The Hobbit", "author": "J.R.R. Tolkien", "genre": "Fantasy",
     "description": "A hobbit embarks on a perilous adventure with dwarves and dragons."}
]

df = pd.DataFrame(data)
df


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

df["embeddings"] = df["description"].apply(lambda x: model.encode(x))
df.head()


In [ ]:
import faiss

# Create the index
dim = len(df["embeddings"][0])
index = faiss.IndexFlatIP(dim)  # cosine similarity via dot product
emb_matrix = np.vstack(df["embeddings"].values)
faiss.normalize_L2(emb_matrix)
index.add(emb_matrix)

# Function to get top-N similar books
def recommend(title, top_n=3):
    idx = df.index[df["title"] == title][0]
    query = df.loc[idx, "embeddings"].reshape(1, -1)
    faiss.normalize_L2(query)
    scores, indices = index.search(query, top_n + 1)
    results = df.iloc[indices[0][1:]][["title", "author", "genre"]]
    results["score"] = scores[0][1:]
    return results

recommend("The Hobbit")
